In [ ]:
import numpy as np
import pandas as pd

def clean_df(df):
    cols = [
        "unique_id",
        "cluster",
        "full_name",
        "dob",
        "birth_place",
        "postcode",
        "gender",
        "occupation",
    ]

    df = df[cols].copy()

    df["name_split"] = df["full_name"].str.strip().str.split(" ")
    df["name_split_length"] = df["name_split"].str.len()
    df["first_name"] = df["name_split"].str[0]
    df["surname"] = df["name_split"].str[-1]
    df["surname"] = np.where(df["name_split_length"] > 1, df["surname"], "")
    # df["middle_names"] = df["name_split"].str[1:-1]

    df["first_and_surname"] = df["first_name"] + " " + df["surname"]

    for col in [
        "full_name",
        "first_and_surname",
        "first_name",
        "surname",
        "dob",
        "birth_place",
        "postcode",
        "gender",
        "occupation",
    ]:
        df[col] = df[col].str.lower().str.strip()
        df[col] = df[col].replace({"": None})

    cols = [
        "unique_id",
        "cluster",
        "full_name",
        "first_and_surname",
        "first_name",
        "surname",
        "dob",
        "birth_place",
        "postcode",
        "gender",
        "occupation",
    ]
    return df[cols]




In [ ]:
import json

from splink.duckdb.duckdb_linker import DuckDBLinker

with open("1m_settings.json", "r") as f:
    settings_dict_loaded = json.load(f)

linker = DuckDBLinker(settings_dict_loaded, connection="1m_compressed.duckdb")

### Compare two records interactively using ipywidgets

In [ ]:
import ipywidgets as widgets

fields = {
    "unique_id": 1,
    "full_name": "james houghton",
    "birth_place": "birmingham",
    "dob": "1985-02-19",
    "postcode": "SW1A 1AA",
    "gender": "male",
    "occupation": "plumber",
    "cluster": 2,
}

left_text_boxes = []
right_text_boxes = []

inputs_to_interactive_output = {}

for key, value in fields.items():
    wl = widgets.Text(description=key, value=str(value))
    left_text_boxes.append(wl)
    inputs_to_interactive_output[f"{key}_l"] = wl
    wr = widgets.Text(description=key, value=str(value))
    right_text_boxes.append(wr)
    inputs_to_interactive_output[f"{key}_r"] = wr


b1 = widgets.VBox(left_text_boxes)
b2 = widgets.VBox(right_text_boxes)
ui = widgets.HBox([b1, b2])


def myfn(**kwargs):
    my_args = dict(kwargs)

    record_left = {}
    record_right = {}

    for key, value in my_args.items():
        if value == "":
            value = None
        if key.endswith("_l"):
            record_left[key[:-2]] = value
        if key.endswith("_r"):
            record_right[key[:-2]] = value

    record_left = clean_df(pd.DataFrame([record_left])).to_dict(orient="records")[0]
    record_right = clean_df(pd.DataFrame([record_right])).to_dict(orient="records")[0]

    linker.settings_obj._retain_intermediate_calculation_columns = True
    linker.settings_obj._retain_matching_columns = True

    df_two = linker.compare_two_records(record_left, record_right)
    df_two.as_pandas_dataframe()
    recs = df_two.as_pandas_dataframe().to_dict(orient="records")
    from splink.charts import waterfall_chart

    waterfall_chart(recs, linker.settings_obj, filter_nulls=False)


out = widgets.interactive_output(myfn, inputs_to_interactive_output)

display(ui, out)

## Search for existing records

In [ ]:
import ipywidgets as widgets

from splink.charts import waterfall_chart

pd.options.display.max_columns = 1000


@widgets.interact(
    full_name="Rudyard Kipling",
    birth_place="Mumbai",
    dob="1865-12-30",
    postcode="B95 5DG",
    gender="male",
    occupation="writer",
)
def interactive_link(full_name, birth_place, dob, postcode, gender, occupation):

    linker.settings_obj._retain_intermediate_calculation_columns = False
    linker.settings_obj._retain_matching_columns = True

    record = {
        "unique_id": 123987,
        "full_name": full_name,
        "birth_place": birth_place,
        "dob": dob,
        "postcode": postcode,
        "gender": gender,
        "occupation": occupation,
        "cluster": 1,
    }

    record = clean_df(pd.DataFrame([record])).to_dict(orient="records")[0]

    if record["full_name"] is not None and len(record["full_name"]) > 4:
        df_inc = linker.find_matches_to_new_records(
            [record], blocking_rules=["l.first_name = r.first_name", "l.dob = r.dob"], match_weight_threshold=-100
        ).as_pandas_dataframe()
        df_inc = df_inc.sort_values("match_weight", ascending=False).head(50)

        if len(df_inc) == 0:
            display("nothing found")
        else:
            recs = df_inc.to_dict(orient="records")

            waterfall_chart(recs, linker.settings_obj)

            cols = list(df_inc.columns)
            cols = [c for c in cols if c.endswith("_l")]
            cols.insert(0, "match_weight")
            cols.insert(0, "match_probability")
            display(df_inc[cols])


## Appendix: How the model was trained:

In [ ]:
df_orig = pd.read_parquet("./benchmarking/synthetic_data_all.parquet")


df_cleaned = clean_df(df_orig)

In [ ]:
import pandas as pd

from splink.comparison_levels_library import (
    else_level,
    exact_match_level,
    levenshtein_level,
    null_level,
)

full_name_cc = {
    "column_name": "full_name",
    "comparison_levels": [
        null_level("full_name"),
        {
            "sql_condition": "full_name_l = full_name_r",
            "label_for_charts": "Full name exact",
            "tf_adjustment_column": "full_name",
            "tf_adjustment_weight": 1.0,
        },
        {
            "sql_condition": "first_and_surname_l = first_and_surname_r",
            "tf_adjustment_column": "first_and_surname",
            "tf_adjustment_weight": 1.0,
            "label_for_charts": "first name surname exact",
        },
        {
            "sql_condition": "levenshtein(full_name_l, full_name_r) <= 2",
            "label_for_charts": "Full name Levenstein <= 2",
            "tf_adjustment_column": "full_name",
            "tf_adjustment_weight": 0.6,
        },
        {
            "sql_condition": "first_name_l = first_name_r",
            "tf_adjustment_column": "first_name",
            "tf_adjustment_weight": 1.0,
            "label_for_charts": "first name exact",
        },
        {
            "sql_condition": "surname_l = surname_r",
            "tf_adjustment_column": "surname",
            "tf_adjustment_weight": 1.0,
            "label_for_charts": "surname exact",
        },
        {
            "sql_condition": "levenshtein(full_name_l, full_name_r) <= 4",
            "label_for_charts": "Full name Levenstein <= 4",
            "tf_adjustment_column": "full_name",
            "tf_adjustment_weight": 0.4,
        },
        {
            "sql_condition": "levenshtein(full_name_l, full_name_r) <= 6",
            "label_for_charts": "Full name Levenstein <= 6",
        },
        else_level(),
    ],
}

dob_cc = {
    "column_name": "dob",
    "comparison_levels": [
        null_level("dob"),
        exact_match_level("dob", term_frequency_adjustments=True),
        levenshtein_level("dob", 1),
        levenshtein_level("dob", 2),
        else_level(),
    ],
}

birth_place_cc = {
    "column_name": "birth_place",
    "comparison_levels": [
        null_level("birth_place"),
        exact_match_level("birth_place", term_frequency_adjustments=True),
        else_level(),
    ],
}

postcode_cc = {
    "column_name": "postcode",
    "comparison_levels": [
        null_level("postcode"),
        exact_match_level("postcode", term_frequency_adjustments=True),
        levenshtein_level("postcode", 2),
        else_level("postcode"),
    ],
}

occupation_cc = {
    "column_name": "occupation",
    "comparison_levels": [
        null_level("occupation"),
        exact_match_level("occupation", term_frequency_adjustments=True),
        else_level(),
    ],
}

In [ ]:
settings_dict = {
    "proportion_of_matches": 0.0001,
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.postcode = r.postcode and l.first_name = r.first_name",
        "l.first_name = r.first_name and l.surname = r.surname",
        "l.dob = r.dob and substr(l.postcode,1,2) = substr(r.postcode,1,2)",
        "l.postcode = r.postcode and substr(l.dob,1,3) = substr(r.dob,1,3)",
        "l.postcode = r.postcode and substr(l.dob,4,5) = substr(r.dob,4,5)",
    ],
    "comparisons": [
        full_name_cc,
        dob_cc,
        birth_place_cc,
        postcode_cc,
        occupation_cc,
    ],
    "retain_matching_columns": False,
    "retain_intermediate_calculation_columns": False,
    "additional_columns_to_retain": ["cluster"],
    "max_iterations": 20,
}

from splink.duckdb.duckdb_linker import DuckDBLinker

linker = DuckDBLinker(
    settings_dict, input_tables={"main": df_cleaned}, connection=":temporary:"
)

In [ ]:
# Issue is
linker.compute_tf_table("full_name")
linker.compute_tf_table("first_name")
linker.compute_tf_table("surname")
linker.compute_tf_table("first_and_surname")
linker.compute_tf_table("postcode")
linker.compute_tf_table("occupation")
linker.compute_tf_table("birth_place")
linker.compute_tf_table("postcode")
linker.compute_tf_table("dob")
linker.train_u_using_random_sampling(target_rows=1e7)

linker.train_m_using_expectation_maximisation("l.full_name = r.full_name")

linker.train_m_using_expectation_maximisation(
    "l.first_name = r.first_name and substr(l.postcode,1,3) = substr(r.postcode,1,3)"
)
linker.train_m_using_expectation_maximisation(
    "l.surname = r.surname and substr(l.dob,1,6) = substr(r.dob,1,6)"
)

linker.train_m_using_expectation_maximisation(
    "l.dob = r.dob and substr(l.postcode,1,2) = substr(r.postcode,1,2)"
)

linker.settings_obj.match_weights_chart()

In [ ]:
import json

with open("1m_settings.json", "w") as f:
    json.dump(linker.settings_obj.as_dict, f)
linker.delete_tables_created_by_splink_from_db()
linker.export_to_duckdb_file("1m_compressed.duckdb")